### Setting up Training Data ###

In [ ]:
from __future__ import print_function
import numpy as np
import h5py
import csv
import matplotlib.pyplot as plt
import torch
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torchvision
from torchvision import datasets, transforms
with h5py.File("data.hdf5", 'r') as f:
    train_data = f['train'][()]
    test_data = f['test'][()]
labels = open('train_labels.csv')
labels_reader = csv.reader(labels)
classes = ["FM","OQPSK","BPSK","8PSK","AM-SSB-SC","4ASK","16PSK","AM-DSB-SC","QPSK","OOK"]
train_labels_data = []
for row in labels_reader:
    if row[0] != 'Id':
        train_labels_data.append(classes.index(row[1]))

train_labels_data = np.asarray(train_labels_data)
print(train_data.shape)
print(len(train_labels_data))
print(test_data.shape)

### Data Augmentation and Training/Validation Set Processing ###

In [ ]:

counter = 0
for i in train_data:
    print(counter)
    if counter > 4000:
        break
    # rotation matrix
    theta = np.pi/2
    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    np.append(train_data,np.dot(rot,i.transpose()).transpose())
    np.append(train_labels_data,train_labels_data[counter])
    
    theta = np.pi
    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    np.append(train_data,np.dot(rot, i.transpose()).transpose())
    np.append(train_labels_data, train_labels_data[counter])
    counter += 1
    
train_data

In [ ]:
# converting training images into torch format
train_x = train_data.reshape(30000, 1, 1024, 2)
train_x  = torch.from_numpy(train_x)

# train_fft_x = fft_train_2d.reshape(30000, 1, 1024, 2)
# train_fft_x  = torch.from_numpy(train_fft_x)

# converting test images into torch format
test_x = test_data.reshape(20000, 1, 1024, 2)
test_x  = torch.from_numpy(test_x)

# converting the target into torch format
train_y = train_labels_data.astype(int)
train_y = torch.from_numpy(train_y)

# split into batches
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.3,shuffle=True)

train_x1, train_x2, train_y1,train_y2 = train_test_split(train_x, train_y, test_size = 0.50,shuffle=True)
train_x3, train_x4, train_y3,train_y4 = train_test_split(train_x1, train_y1, test_size = 0.50,shuffle=True)
train_x5, train_x6, train_y5,train_y6 = train_test_split(train_x2, train_y2, test_size = 0.7,shuffle=True)
train_val1, train_val2, train_valy1,train_valy2 = train_test_split(val_x, val_y, test_size = 0.95,shuffle=True)
# batches
train = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(train, batch_size=240, shuffle=True)

# shape of training data
print(train_x.shape)
print(train_y.shape)

### Visualizing Data  ###

In [ ]:
for j in range(0,50):
    I = []
    Q = []
    for i in range(0,1024):
        I.append(train_data[j][i][0])
        Q.append(train_data[j][i][1])
    plt.figure(j)
    plt.scatter(I,Q)

### Setting up Convolutional Neural Network Architecture ###

In [ ]:
import torch.nn as nn

class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 256, kernel_size=(3,1), stride=1)
        self.batchnorm1 = nn.BatchNorm2d(num_features=256)
        self.batchnorm2 = nn.BatchNorm2d(num_features=80)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=(3,2), stride=1)
        self.conv3 = nn.Conv2d(256, 80, kernel_size=(3,1), stride=1)
        self.conv4 = nn.Conv2d(80, 80, kernel_size=(3,1), stride=1)
        self.dropout = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(81280, 128)
        self.fc2 = nn.Linear(128, len(classes))

    def forward(self, x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = F.relu(self.batchnorm1(self.conv2(x)))
        x = F.selu(self.batchnorm2(self.conv3(x)))
        x = F.selu(self.batchnorm2(self.conv4(x)))
        x = x.view(-1, 81280)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        
cnn = Net2()
print(cnn)

it = iter(trainloader)

x = torch.randn(3,1,1024,2)
conv1 = nn.Conv2d(1, 256, kernel_size=(3,1), stride=1)
y=conv1(x)
print(y.shape)
conv2 = nn.Conv2d(256, 256, kernel_size=(3,2), stride=1)
y=conv2(y)
print(y.shape)
conv3 = nn.Conv2d(256, 80, kernel_size=(3,1), stride=1)
conv4 = nn.Conv2d(80, 80, kernel_size=(3,1), stride=1)
y=conv3(y)
print(y.shape)
y=conv4(y)
print(y.shape)

# Set up training protocol ###

In [ ]:
def fit(model, train_loader):
    optimizer = torch.optim.Adagrad(model.parameters(),lr=0.0002, weight_decay=0.00009 )
    error = nn.CrossEntropyLoss()
    EPOCHS = 10000
    model.train()
    for epoch in range(EPOCHS):
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()
            predicted = torch.max(output.data, 1)[1] 
            print("Accuracy:")
            print(((predicted == var_y_batch).sum()).double()/240)
            print("Loss:")
            print(loss)
            print("Epoch:")
            print(epoch)
            print("Batch:")
            print(batch_idx)
            if (batch_idx % 20 == 0):
                torch.save(model.state_dict(), 'weights.pt')
                with torch.no_grad():
                    validout = model(train_val1)
                valpred = torch.max(validout.data, 1)[1] 
                print("########################")
                print("Validation Accuracy:")
                print(((valpred == train_valy1).sum()).double()/len(train_valy1))
                print("########################")

In [ ]:
fit(cnn, trainloader)

### Predictions ###

In [ ]:
with torch.no_grad():
    output = cnn(test_x)

In [ ]:
prob = output.tolist()
predictions = np.argmax(prob, axis=1)

pred_label = []
for i in predictions:
    pred_label.append(classes[i])
from numpy import asarray
from numpy import savetxt
import pandas as pd
# define id
id = np.arange(0,20000)
labels=np.asarray(pred_label)
labels.reshape(20000,1)
print(labels.shape)
d = {'Category':labels}
df = pd.DataFrame(data=d)
df.to_csv('submission.csv')